# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
#your code here
from sklearn.preprocessing import StandardScaler

# Selecting the numerical columns to scale (you'll typically avoid columns like 'PassengerId')
numerical_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the data
spaceship[numerical_features] = scaler.fit_transform(spaceship[numerical_features])

# Check the scaled data
print(spaceship[numerical_features].head())


        Age  RoomService  FoodCourt  ShoppingMall       Spa    VRDeck
0  0.702095    -0.337025  -0.284274     -0.287317 -0.273736 -0.266098
1 -0.333233    -0.173528  -0.278689     -0.245971  0.209267 -0.227692
2  2.013510    -0.272527   1.934922     -0.287317  5.634034 -0.223327
3  0.287964    -0.337025   0.511931      0.326250  2.655075 -0.097634
4 -0.885407     0.117466  -0.240833     -0.037590  0.223344 -0.264352


In [8]:
#your code here
features = spaceship[numerical_features][['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']]
target = spaceship['Transported']

**Perform Train Test Split**

In [5]:
from sklearn.model_selection import train_test_split

# Assuming 'features' is your feature matrix and 'target' is your target variable
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Check the shapes of the resulting splits
print(f"Training Features Shape: {X_train.shape}")
print(f"Test Features Shape: {X_test.shape}")
print(f"Training Target Shape: {y_train.shape}")
print(f"Test Target Shape: {y_test.shape}")


Training Features Shape: (6954, 6)
Test Features Shape: (1739, 6)
Training Target Shape: (6954,)
Test Target Shape: (1739,)


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [10]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

# Handle missing values using SimpleImputer (mean strategy)
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Bagging Classifier with Decision Trees
bagging_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500, max_samples=100, bootstrap=True, random_state=42
)

# Train the model
bagging_clf.fit(X_train_imputed, y_train)

# Make predictions
y_pred_bagging = bagging_clf.predict(X_test_imputed)

# Evaluate the performance
print("Bagging Accuracy:", accuracy_score(y_test, y_pred_bagging))


Bagging Accuracy: 0.7705577918343876


- Random Forests

In [12]:
#your code here
from sklearn.ensemble import RandomForestClassifier

# Impute missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')  # You can also try 'median', 'most_frequent', etc.
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=500, random_state=42)

# Train the model
rf_clf.fit(X_train_imputed, y_train)

# Make predictions
y_pred_rf = rf_clf.predict(X_test_imputed)

# Evaluate the performance
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))


Random Forest Accuracy: 0.7711328349626222


- Gradient Boosting

In [15]:
#your code here
from sklearn.ensemble import GradientBoostingClassifier

# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, random_state=42)

# Train the model
gb_clf.fit(X_train_imputed, y_train)

# Make predictions
y_pred_gb = gb_clf.predict(X_test_imputed)

# Evaluate the performance
print("Gradient Boosting Accuracy:", accuracy_score(y_test, y_pred_gb))


Gradient Boosting Accuracy: 0.7780333525014376


- Adaptive Boosting

Which model is the best and why?

In [ ]:
#comment here